In [31]:
import pandas as pd
import numpy as np
import re

In [36]:
base_url = "C:/Users/HP/Desktop/laptop_price_prediction_project/data/"
combined_data = pd.read_csv(base_url+'dirty_data.csv')
clean_data.head()

,reference,link,name,brand,image_url,store,screen_size,processor,ram,disk_capacity,gpu,os,gamer,price
0,X515KA-EJ008,https://agora.tn/fr/pc-portable/16616-pc-porta...,PC PORTABLE ASUS X515KA CELERON N4500U 4GO 256...,Asus,https://agora.tn/fr/46133-home_default/pc-port...,agora,15.6,Intel Celeron,4,256 Go SSD,Graphique Intégrée,FreeDos,Non,719.0
1,X515KA-EJ008-8,https://agora.tn/fr/pc-portable/16664-pc-porta...,PC PORTABLE ASUS X515KA CELERON N4500U 8GO 256...,Asus,https://agora.tn/fr/46726-home_default/pc-port...,agora,15.6,Intel Celeron,8,256 Go SSD,Graphique Intégrée,FreeDos,Non,739.0
2,X1504VA-NJ520W,https://agora.tn/fr/pc-portable/15389-pc-porta...,PC PORTABLE ASUS VIVOBOOK 15 X1504VA I3-1315U ...,Asus,https://agora.tn/fr/46316-home_default/pc-port...,agora,15.6,Intel Core i3,4,256 Go SSD,Graphique Intégrée,Windows,Non,1049.0
3,INS-3535-R5,https://agora.tn/fr/pc-portable/16867-pc-porta...,PC PORTABLE DELL INSPIRON 3535 AMD RAYZEN 5 8G...,Dell,https://agora.tn/fr/46775-home_default/pc-port...,agora,15.6,AMD RYZEN 5,8,512 Go SSD,AMD Radeon,FreeDos,Non,1169.0
4,X1504VA-NJ814W,https://agora.tn/fr/pc-portable/17170-pc-porta...,PC PORTABLE ASUS VIVOBOOK 15 I3-1315U 8GO 512G...,Asus,https://agora.tn/fr/48086-home_default/pc-port...,agora,15.6,Intel Core i3,8,512 Go SSD,Graphique Intégrée,Windows 11,Non,1189.0


In [35]:
combined_data.info()

<class 'pandas.DataFrame'>
RangeIndex: 3931 entries, 0 to 3930
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   reference      3927 non-null   object
 1   link           3931 non-null   object
 2   name           3931 non-null   object
 3   brand          2911 non-null   object
 4   image_url      3931 non-null   object
 5   store          3931 non-null   object
 6   screen_size    3908 non-null   object
 7   processor      3922 non-null   object
 8   ram            3924 non-null   object
 9   disk_capacity  3899 non-null   object
 10  gpu            3878 non-null   object
 11  os             3930 non-null   object
 12  gamer          3190 non-null   object
 13  price          3931 non-null   object
dtypes: object(14)
memory usage: 430.1+ KB


In [21]:
# List of possible brand names
brand_keywords = ['apple', 'asus', 'dell', 'gigabyte', 'hp', 'lenovo', 'msi', 'schneider', 'thomson', 'acer']

# Function to fill missing brand values based on the name column
def detect_brand(name):
    name = name.lower()
    for brand in brand_keywords:
        if brand in name:
            return brand.capitalize()
    return None

# Replace 'brand' column with the detected brand for each dataframe
clean_data['brand'] = clean_data['name'].apply(detect_brand)

In [22]:
# Function to standardize all screen sizes to inches
def remove_text_after_special_chars(text):
    if text :
        return re.sub(r'[-\(|"].*', '', str(text))

def standardize_screen_size(text):
    text = re.sub(r',', '.', str(text))
    text = re.sub(r'[^0-9.]', '', text)
    if text == '':
        return np.nan
    return float(text)
    

clean_data['screen_size'] = clean_data['screen_size'].apply(remove_text_after_special_chars)

clean_data['screen_size'] = clean_data['screen_size'].apply(standardize_screen_size)

In [23]:
def remove_text_after_special_chars(text):
    if text :
        return re.sub(r'[-\(|].*', '', str(text))

def remove_special_characters(text):
    if text:
        return re.sub(r'[^\w\s]', '', text)

def remove_invalid_processor_rows(df):
    df['processor'] = df['processor'].replace('nan', np.nan)
    return df.dropna(subset=['processor'])


#remove_text_after_special_chars
clean_data['processor'] = clean_data['processor'].apply(remove_text_after_special_chars)

#remove_special_characters
clean_data['processor'] = clean_data['processor'].apply(remove_special_characters)

clean_data = remove_invalid_processor_rows(clean_data)

In [24]:
def clean_ram_column(text):
    if text :
        text = re.sub(r',', '.', str(text))
        cleaned_text = re.sub(r'\D', '', str(text)) 
    return cleaned_text

clean_data['ram'] = clean_data['ram'].apply(clean_ram_column)

In [25]:
def add_space(text):
    if text :
        text = re.sub(r'GO', 'Go', str(text))
        text = re.sub(r"(\d)([A-Za-z])", r"\1 \2", text)
    return text

clean_data['disk_capacity'] = clean_data['disk_capacity'].apply(add_space)

In [26]:
def clean_price_column(text):
    text = re.sub(r',', '.', str(text))
    cleaned_text = re.sub(r'[^\d.]', '', str(text))
    return float(cleaned_text)

clean_data['price'] = clean_data['price'].apply(clean_price_column)

In [27]:
def remove_invalid_rows(df):
    columns_to_check = ['screen_size', 'processor', 'ram', 'disk_capacity', 'gpu', 'os', 'gamer', 'price']
    
    for col in columns_to_check:
        df[col] = df[col].replace('nan', np.nan)

    df = df.dropna(subset=columns_to_check)
    return df

clean_data = remove_invalid_rows(clean_data)

In [28]:
clean_data = clean_data.drop_duplicates(subset=['store','brand','screen_size', 'processor', 'ram', 'disk_capacity', 'gpu', 'os', 'gamer', 'price'])

In [29]:
len(clean_data)

1890

In [137]:
clean_data.head(2)

,reference,link,name,brand,image_url,store,screen_size,processor,ram,disk_capacity,gpu,os,gamer,price
0,X515KA-EJ008,https://agora.tn/fr/pc-portable/16616-pc-porta...,PC PORTABLE ASUS X515KA CELERON N4500U 4GO 256...,Asus,https://agora.tn/fr/46133-home_default/pc-port...,agora,15.6,Intel Celeron,4,256 Go SSD,Graphique Intégrée,FreeDos,Non,719.0
1,X515KA-EJ008-8,https://agora.tn/fr/pc-portable/16664-pc-porta...,PC PORTABLE ASUS X515KA CELERON N4500U 8GO 256...,Asus,https://agora.tn/fr/46726-home_default/pc-port...,agora,15.6,Intel Celeron,8,256 Go SSD,Graphique Intégrée,FreeDos,Non,739.0


In [139]:
# Save to CSV
clean_data.to_csv("clean_data.csv", index=False, encoding='utf-8-sig')

print("Data saved to clean_data.csv")

Data saved to clean_data.csv
